In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from datetime import timedelta

In [5]:
df = pd.read_csv("Amazon.csv")
df.head()


,OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Category,Brand,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount,PaymentMethod,OrderStatus,City,State,Country,SellerID
0,ORD0000001,2023-01-31,CUST001504,Vihaan Sharma,P00014,Drone Mini,Books,BrightLux,3,106.59,0.00,0.00,0.09,319.86,Debit Card,Delivered,Washington,DC,India,SELL01967
1,ORD0000002,2023-12-30,CUST000178,Pooja Kumar,P00040,Microphone,Home & Kitchen,UrbanStyle,1,251.37,0.05,19.10,1.74,259.64,Amazon Pay,Delivered,Fort Worth,TX,United States,SELL01298
2,ORD0000003,2022-05-10,CUST047516,Sneha Singh,P00044,Power Bank 20000mAh,Clothing,UrbanStyle,3,35.03,0.10,7.57,5.91,108.06,Debit Card,Delivered,Austin,TX,United States,SELL00908
3,ORD0000004,2023-07-18,CUST030059,Vihaan Reddy,P00041,Webcam Full HD,Home & Kitchen,Zenith,5,33.58,0.15,11.42,5.53,159.66,Cash on Delivery,Delivered,Charlotte,NC,India,SELL01164
4,ORD0000005,2023-02-04,CUST048677,Aditya Kapoor,P00029,T-Shirt,Clothing,KiddoFun,2,515.64,0.25,38.67,9.23,821.36,Credit Card,Cancelled,San Antonio,TX,Canada,SELL01411


In [6]:
print(df.columns)



Index(['OrderID', 'OrderDate', 'CustomerID', 'CustomerName', 'ProductID',
       'ProductName', 'Category', 'Brand', 'Quantity', 'UnitPrice', 'Discount',
       'Tax', 'ShippingCost', 'TotalAmount', 'PaymentMethod', 'OrderStatus',
       'City', 'State', 'Country', 'SellerID'],
      dtype='object')


In [7]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
print(df.columns)


Index(['orderid', 'orderdate', 'customerid', 'customername', 'productid',
       'productname', 'category', 'brand', 'quantity', 'unitprice', 'discount',
       'tax', 'shippingcost', 'totalamount', 'paymentmethod', 'orderstatus',
       'city', 'state', 'country', 'sellerid'],
      dtype='object')


In [9]:
df['orderdate'] = pd.to_datetime(df['orderdate'])


In [10]:
df['revenue'] = df['quantity'] * df['unitprice']


In [13]:
rfm = df.groupby('customerid').agg({
    'orderdate': 'max',      # Last purchase date
    'orderid': 'nunique',    # Frequency
    'revenue': 'sum'          # Monetary
}).reset_index()



In [14]:
rfm.columns = ['customer_id', 'last_purchase_date', 'frequency', 'monetary']


In [15]:
analysis_date = rfm['last_purchase_date'].max() + pd.Timedelta(days=1)

rfm['recency'] = (analysis_date - rfm['last_purchase_date']).dt.days


In [16]:
rfm.head()
rfm.describe()


,last_purchase_date,frequency,monetary,recency
count,43233,43233.000000,43233.000000,43233.000000
mean,2023-04-11 20:01:40.922906112,2.313048,2105.364259,628.165499
min,2020-01-01 00:00:00,1.000000,5.150000,1.000000
25%,2022-05-21 00:00:00,1.000000,876.420000,225.000000
50%,2023-07-30 00:00:00,2.000000,1784.630000,519.000000
75%,2024-05-19 00:00:00,3.000000,2957.160000,954.000000
max,2024-12-29 00:00:00,10.000000,14558.680000,1825.000000
std,NaN,1.257115,1577.239778,478.122067


In [18]:
rfm['r_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])


In [21]:
rfm['r_score'] = rfm['r_score'].astype(int)
rfm['f_score'] = rfm['f_score'].astype(int)
rfm['m_score'] = rfm['m_score'].astype(int)



In [22]:
def segment_customer(row):
    if row['r_score'] >= 4 and row['f_score'] >= 4 and row['m_score'] >= 4:
        return 'Champions'
    elif row['r_score'] >= 3 and row['f_score'] >= 3:
        return 'Loyal Customers'
    elif row['r_score'] <= 2 and row['f_score'] >= 3:
        return 'At Risk'
    else:
        return 'Others'

rfm['segment'] = rfm.apply(segment_customer, axis=1)



In [23]:
rfm.groupby('segment')['monetary'].mean().sort_values(ascending=False)


segment
Champions          4071.712161
At Risk            2402.554426
Loyal Customers    2195.093956
Others             1111.590804
Name: monetary, dtype: float64

In [24]:
rfm['clv'] = rfm['monetary'] * rfm['frequency']


In [25]:
rfm.to_csv("rfm_output.csv", index=False)


In [26]:
rfm['segment'].value_counts()


segment
Others             17293
Loyal Customers    11616
At Risk             7203
Champions           7121
Name: count, dtype: int64